In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import random
import os

In [ ]:
train_df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-1/train.csv', encoding='ISO-8859-1')
# Define path to video clips
video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-1/train_videos'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply the function to get file paths for each sampled clip
train_df['video_clip_path'] = train_df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(train_df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

In [ ]:
train_df.shape

In [ ]:
# Define path to video clips
df = pd.read_csv('/kaggle/input/ml-hackathon-ec-campus-set-1/test.csv', encoding='ISO-8859-1')
video_dir = '/kaggle/input/ml-hackathon-ec-campus-set-1/test_videos'


# Function to get video file path from IDs
def get_video_clip_path(row):
    dialogue_id = row['Dialogue_ID']
    utterance_id = row['Utterance_ID']
    filename = f"dia{dialogue_id}_utt{utterance_id}.mp4"
    return os.path.join(video_dir, filename)

# Apply the function to get file paths for each sampled clip
df['video_clip_path'] = df.apply(get_video_clip_path, axis=1)

# Check sample paths
print(df[['Dialogue_ID', 'Utterance_ID', 'video_clip_path']].head())

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.stattools import adfuller, kpss

In [ ]:
#print(train_df.head())
sentences = train_df[["Utterance","Sentiment"]].values
sentences = pd.DataFrame(sentences)

sentences_test = df["Utterance"].values
sentences_test = pd.DataFrame(sentences_test)

sentences.columns = ['Text', 'Sentiment']
print(sentences)

sentences_test.columns = ['Text']
print(sentences_test)

In [ ]:
'''from transformers import pipeline, BertModel

# Use a model with 3 sentiment classes
model = BertModel.from_pretrained
classifier = pipeline("sentiment-analysis", model="cardiffnlp/twitter-roberta-base-sentiment")
results = classifier(sentences)

# Display results
for sentence, result in zip(sentences, results):
    print(f"Sentence: {sentence}\nSentiment: {result['label']} (Score: {result['score']:.2f})\n")'''

In [ ]:
'''results_df = pd.DataFrame(results)
results_df['label'] = results_df['label'].map({'LABEL_2':1,'LABEL_0':2,'LABEL_1':2})
print(results_df.head())'''

In [ ]:
'''from sklearn.model_selection import train_test_split

# Features (confidence score) and target (label)
X = results_df[['score']]
y = results_df['label']

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)'''

In [ ]:
'''from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, classification_report

# Initialize the MLP model
mlp_model = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', max_iter=500, random_state=42)

# Train the MLP model
mlp_model.fit(X_train, y_train)

# Predict on the test data
y_pred = mlp_model.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))'''

In [ ]:
'''import torch
from transformers import BertTokenizer, BertForSequenceClassification
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

model_name = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(model_name)
model = BertForSequenceClassification.from_pretrained(model_name, num_labels=3)

# Clean text data by removing stopwords and extra spaces
sentences['Text'] = sentences['Text'].apply(lambda x: ' '.join(
    [word for word in x.split() if word.lower() not in ENGLISH_STOP_WORDS])
)
# Check if any rows are empty after cleaning
empty_text = sentences[sentences['Text'].str.strip() == '']
print(f"Number of empty text rows: {empty_text.shape[0]}")'''

In [ ]:
'''from sklearn.feature_extraction.text import TfidfVectorizer

# Using TfidfVectorizer to convert text data to numerical features
vectorizer = TfidfVectorizer(max_features=5000)  # Limit to 5000 features for efficiency
X = vectorizer.fit_transform(sentences['Text'])

# Labels
y = sentences['Sentiment']

# Split into training and testing datasets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Initialize Random Forest model
from sklearn.ensemble import RandomForestClassifier
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)

# Train the Random Forest model
rf_model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test)

# Evaluate the model
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))'''

In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Example DataFrame (replace with your actual sentences)
# sentences = pd.DataFrame({'Text': ["I love this!", "I hate that", "This is okay"], 'Sentiment': ['positive', 'negative', 'neutral']})

# Encode sentiments into numeric labels
sentences['Sentiment'] = sentences['Sentiment'].map({'positive': 1, 'negative': 0, 'neutral': 2})

# Split the data into training and testing sets
train_data, test_data = train_test_split(sentences, test_size=0.3, random_state=42)

# Load BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Create custom Dataset class
class SentimentDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = sentences['Text'].values
        self.labels = sentences['Sentiment'].values

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        label = self.labels[idx]
        
        # Encode text using BERT tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Prepare DataLoader
train_dataset = SentimentDataset(train_data, tokenizer)
test_dataset = SentimentDataset(test_data, tokenizer)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Load pre-trained BERT model for sequence classification
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)

# Move model to GPU if available
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

# Optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Training loop
epochs = 3
for epoch in range(epochs):
    model.train()
    for batch in train_dataloader:
        optimizer.zero_grad()
        
        # Move batch to GPU if available
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        
        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        
        # Optimizer step
        optimizer.step()

    print(f"Epoch {epoch + 1}/{epochs} completed.")

# Evaluation
model.eval()
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

# Print classification report for 3 sentiment classes
print(classification_report(true_labels, predictions, target_names=['negative', 'positive', 'neutral']))

In [ ]:
# Assuming `test_data` contains only 'Text' column without the 'Sentiment' column
# Replace the previous evaluation section with the following code

test_sentences = sentences_test['Text'].values  # Only the text for prediction

# Create a dataset for test data without labels
class TestSentimentDataset(Dataset):
    def __init__(self, sentences, tokenizer, max_length=128):
        self.sentences = sentences
        self.tokenizer = tokenizer
        self.max_length = max_length
        self.texts = sentences

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        
        # Encode text using BERT tokenizer
        encoding = self.tokenizer.encode_plus(
            text,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=self.max_length,
            padding='max_length',
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )
        
        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten()
        }

# Prepare DataLoader for test data
test_dataset = TestSentimentDataset(test_sentences, tokenizer)
test_dataloader = DataLoader(test_dataset, batch_size=16)

# Make predictions on the test dataset
model.eval()
predictions = []
with torch.no_grad():
    for batch in test_dataloader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        preds = torch.argmax(logits, dim=1)

        predictions.extend(preds.cpu().numpy())

# Map predictions back to sentiment labels
sentiment_labels = ['negative', 'positive', 'neutral']
predicted_sentiments = [sentiment_labels[pred] for pred in predictions]

all_preds = predicted_sentiments

submission_df = pd.DataFrame({
        'ID': range(1,len(test_sentences)+1),
        'Emotion': all_preds
    })
    
# Save the DataFrame to CSV
submission_df.to_csv("submission.csv", index=False)

In [ ]:
pip install moviepy

In [ ]:
from moviepy.editor import VideoFileClip
import os

# Path to the video directory
video_dir = "/kaggle/input/ml-hackathon-ec-campus-set-1/train"  # Your video folder path

# Create a directory to store audio files
audio_dir = "/kaggle/working/audio_files"
os.makedirs(audio_dir, exist_ok=True)

# Iterate through all video files in the directory
for filename in os.listdir(video_dir):
    if filename.endswith(('.mp4', '.avi', '.mov')):  # Adjust extensions as needed
        video_path = os.path.join(video_dir, filename)
        
        # Load video file
        video_clip = VideoFileClip(video_path)
        
        # Extract audio from the video
        audio_clip = video_clip.audio
        
        # Save the audio file
        audio_filename = os.path.splitext(filename)[0] + '.mp3'  # You can also use .wav
        audio_path = os.path.join(audio_dir, audio_filename)
        audio_clip.write_audiofile(audio_path)
        
        # Close clips after processing to free resources
        video_clip.close()
        audio_clip.close()

print("Audio extraction complete!")

In [ ]:
import librosa
import numpy as np
import os

# Path to your audio files directory
audio_dir = "/kaggle/working/audio_files"  # Update with your path

# Function to extract MFCC features from an audio file
def extract_audio_features(audio_file):
    # Load audio file
    y, sr = librosa.load(audio_file, sr=None)
    
    # Extract MFCC features
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)
    
    # Mean of the MFCC features across time
    mfcc_mean = np.mean(mfcc, axis=1)
    
    return mfcc_mean

# Extract features for all audio files
audio_features = []
audio_filenames = []

for filename in os.listdir(audio_dir):
    if filename.endswith('.mp3'):  # Adjust file extensions as needed
        audio_file_path = os.path.join(audio_dir, filename)
        features = extract_audio_features(audio_file_path)
        audio_features.append(features)
        audio_filenames.append(filename)

# Convert to numpy array
audio_features = np.array(audio_features)


In [ ]:
import pandas as pd

# Load the dataset with ISO-8859-1 encoding
file_path = "/kaggle/input/ml-hackathon-ec-campus-set-1/train.csv"  # Replace with your actual CSV file path
df = pd.read_csv(file_path, encoding='ISO-8859-1')

# Preview the data to check the columns
print(df.head())

# Check for missing values
print(df.isnull().sum())


# Drop rows with missing 'Utterance' or 'Sentiment' columns
df = df.dropna(subset=['Utterance', 'Sentiment'])

# Preview cleaned data
print(df[['Utterance', 'Sentiment']].head())

In [ ]:
from transformers import pipeline

# Initialize the sentiment analysis pipeline using BERT
sentiment_analyzer = pipeline("sentiment-analysis")

# Function to analyze sentiment of the utterance
def analyze_text_sentiment(text):
    result = sentiment_analyzer(text)
    sentiment = result[0]['label']  # "POSITIVE" or "NEGATIVE"
    return sentiment

# Apply sentiment analysis to each utterance
df['Predicted_Sentiment'] = df['Utterance'].apply(analyze_text_sentiment)

# Map BERT sentiment output to numeric labels (positive: 1, negative: 0, neutral: 2)
def sentiment_to_label(sentiment):
    if sentiment == 'POSITIVE':
        return 1
    elif sentiment == 'NEGATIVE':
        return 0
    else:
        return 2  # Neutral

df['Predicted_Sentiment_Label'] = df['Predicted_Sentiment'].apply(sentiment_to_label)

# Preview the updated dataframe
print(df[['Utterance', 'Sentiment', 'Predicted_Sentiment', 'Predicted_Sentiment_Label']].head())

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

# Convert actual sentiment to numeric labels (positive: 1, negative: 0, neutral: 2)
def actual_sentiment_to_label(sentiment):
    if sentiment == 'positive':
        return 1
    elif sentiment == 'negative':
        return 0
    else:
        return 2  # Neutral

# Apply this function to the actual 'Sentiment' column
df['Actual_Sentiment_Label'] = df['Sentiment'].apply(actual_sentiment_to_label)

# Calculate the accuracy
accuracy = accuracy_score(df['Actual_Sentiment_Label'], df['Predicted_Sentiment_Label'])
print(f'Accuracy: {accuracy * 100:.2f}%')

# Generate a classification report
print("Classification Report:\n", classification_report(df['Actual_Sentiment_Label'], df['Predicted_Sentiment_Label']))